In [12]:
import pandas as pd
import os
import pickle
from datetime import datetime
from datetime import timezone

In [14]:
import json

config = json.load(open("../configure.json"))
start_date = datetime.fromisoformat(config['start_time']).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
end_date = datetime.fromisoformat(config['end_time']).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()

In [5]:
charts = os.listdir('data/CHARTS')
length_of_time_windows = [5, 15, 30, 60, 240, 1440]
preferred_time = 240

In [6]:
apple_stock_chart = pd.DataFrame()
amazon_trends = []
amazon_timestamps = []
amazon_time_frames = []
apple_trends = []
apple_timestamps = []
apple_time_frames = []
if 'AMAZON' + str(preferred_time) + ".csv" in charts:
    headers = ["date", "time", "open", "high", "low", "close", "volume"]
    amazon_stock_chart = pd.read_csv('data/CHARTS/' + 'AMAZON' + str(preferred_time) + ".csv", names=headers)
    amazon_trends = amazon_stock_chart.apply(lambda row: 1 if row["close"] > row["open"] else -1, axis=1)
    amazon_timestamps = amazon_stock_chart.apply(
        lambda row: datetime.fromisoformat(row["date"].replace(".", "-") + " " + row["time"]).timestamp(), axis=1)
if 'APPLE' + str(preferred_time) + ".csv" in charts:
    headers = ["date", "time", "open", "high", "low", "close", "volume"]
    apple_stock_chart = pd.read_csv('data/CHARTS/' + 'APPLE' + str(preferred_time) + ".csv", names=headers)
    apple_trends = apple_stock_chart.apply(lambda row: 1 if row["close"] > row["open"] else -1, axis=1)
    apple_timestamps = apple_stock_chart.apply(
        lambda row: datetime.fromisoformat(row["date"].replace(".", "-") + " " + row["time"]).timestamp(), axis=1)

amazon_stock_prices = pd.DataFrame({
    "time": amazon_timestamps,
    "trends": amazon_trends
})
apple_stock_prices = pd.DataFrame({
    "time": apple_timestamps,
    "trends": apple_trends
})
apple_stock_prices.drop(apple_stock_prices[(apple_stock_prices['timestamp'] < start_date) | (
        apple_stock_prices['timestamp'] > end_date)].index,
                        inplace=True)
amazon_stock_prices.drop(amazon_stock_prices[(amazon_stock_prices['timestamp'] < start_date) | (
        amazon_stock_prices['timestamp'] > end_date)].index,
                         inplace=True)
apple_stock_prices.sort_values('timestamp', ascending=True, inplace=True)
amazon_stock_prices.sort_values('timestamp', ascending=True, inplace=True)

In [7]:
amazon_stock_prices

#%

,timestamp,trends
1506,1.514925e+09,1
1507,1.514939e+09,-1
1508,1.514954e+09,1
1509,1.515011e+09,1
1510,1.515026e+09,1
...,...,...
2261,1.548967e+09,1
2262,1.548981e+09,-1
2263,1.548995e+09,1
2264,1.549053e+09,1


In [8]:
apple_stock_prices

,timestamp,trends
2486,1.514925e+09,1
2487,1.514939e+09,1
2488,1.514954e+09,-1
2489,1.515011e+09,1
2490,1.515026e+09,-1
...,...,...
3241,1.548967e+09,1
3242,1.548981e+09,-1
3243,1.548995e+09,1
3244,1.549053e+09,1


In [9]:
with open('pkls/amazon_stock_chart.pkl', 'wb') as f:
    pickle.dump(apple_stock_prices, f)
    print("done")
with open('pkls/apple_stock_chart.pkl', 'wb') as f:
    pickle.dump(apple_stock_prices, f)
    print("done")

done
done
